# Databricks Connect Demo

Simple demonstration of querying and transforming data using Databricks Connect.

**Prerequisites:**
- Databricks Connect configured
- Access to `samples.nyctaxi.trips` table

## 1. Load Data from Databricks

Query the NYC taxi trips sample dataset

In [ ]:
# Read the NYC taxi trips data
df = spark.sql("SELECT * FROM samples.nyctaxi.trips LIMIT 100")

print(f"Loaded {df.count()} rows")
df.show(10)

## 2. Basic Data Exploration

In [ ]:
# Show schema
print("Schema:")
df.printSchema()

# Show sample data
print("\nSample rows:")
df.show(5, truncate=False)

## 3. Select Specific Columns

In [ ]:
from pyspark.sql.functions import col

# Select key columns
trips_subset = df.select(
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "trip_distance",
    "fare_amount",
    "pickup_zip",
    "dropoff_zip"
)

print("Selected columns:")
trips_subset.show(10)

## 4. Add Calculated Columns

Calculate fare per mile and categorize trips by distance

In [ ]:
from pyspark.sql.functions import col, when, round

# Add calculated columns
trips_transformed = trips_subset.withColumn(
    "fare_per_mile",
    round(col("fare_amount") / col("trip_distance"), 2)
).withColumn(
    "trip_category",
    when(col("trip_distance") < 2, "Short")
    .when(col("trip_distance") < 5, "Medium")
    .otherwise("Long")
)

print("Transformed data with calculated columns:")
trips_transformed.show(10)

## 5. Filter and Aggregate

Filter for trips over $10 and calculate summary statistics

In [ ]:
from pyspark.sql.functions import avg, sum, count

# Filter for higher fare trips
high_fare_trips = trips_transformed.filter(col("fare_amount") > 10)

print(f"High fare trips (> $10): {high_fare_trips.count()} trips")
high_fare_trips.show(10)

# Calculate summary statistics by trip category
summary = trips_transformed.groupBy("trip_category").agg(
    count("*").alias("num_trips"),
    avg("fare_amount").alias("avg_fare"),
    avg("trip_distance").alias("avg_distance"),
    sum("fare_amount").alias("total_revenue")
).orderBy("trip_category")

print("\nSummary by trip category:")
summary.show()

## 6. SQL Query Example

In [ ]:
# Create temp view for SQL queries
trips_transformed.createOrReplaceTempView("trips_temp")

# Run SQL query
sql_result = spark.sql("""
    SELECT 
        trip_category,
        COUNT(*) as num_trips,
        ROUND(AVG(fare_amount), 2) as avg_fare,
        ROUND(AVG(fare_per_mile), 2) as avg_fare_per_mile
    FROM trips_temp
    WHERE fare_amount > 5
    GROUP BY trip_category
    ORDER BY avg_fare DESC
""")

print("SQL query results:")
sql_result.show()

## Summary

✅ Successfully queried Databricks data using Databricks Connect  
✅ Selected and filtered columns  
✅ Added calculated columns  
✅ Performed aggregations  
✅ Used SQL queries  